In [ ]:
from google.colab import drive
drive.mount('/gd')

Mounted at /gd


In [ ]:
!pip install transformers sentencepiece sacrebleu datasets  sentence-transformers fasttext -q

     |████████████████████████████████| 4.7 MB 31.2 MB/s 
     |████████████████████████████████| 1.3 MB 64.9 MB/s 
     |████████████████████████████████| 116 kB 74.9 MB/s 
     |████████████████████████████████| 365 kB 73.2 MB/s 
     |████████████████████████████████| 85 kB 5.5 MB/s 
     |████████████████████████████████| 68 kB 8.1 MB/s 
     |████████████████████████████████| 596 kB 6.3 MB/s 
     |████████████████████████████████| 6.6 MB 62.0 MB/s 
     |████████████████████████████████| 101 kB 11.5 MB/s 
     |████████████████████████████████| 141 kB 76.4 MB/s 
     |████████████████████████████████| 115 kB 74.9 MB/s 
     |████████████████████████████████| 212 kB 53.1 MB/s 
     |████████████████████████████████| 127 kB 73.5 MB/s 


In [ ]:
import os
import numpy as np
import pandas as pd
from tqdm.auto import tqdm, trange
import json
import random

In [ ]:
import torch
from transformers import MBartForConditionalGeneration, MBart50Tokenizer

In [ ]:
import gc

def cleanup():
    gc.collect()
    torch.cuda.empty_cache()
    
cleanup()

In [ ]:
BASE_DIR = '/gd/MyDrive/models/myv/'

# Do the manual scores correlate with what the BERT detects? No.

In [ ]:
manual_scores = pd.read_csv(BASE_DIR + 'manual_scores_v1.tsv', sep='\t')

In [ ]:
manual_scores

,original,translation,score,source
0,"Благодарение Богу, вложившему в сердце Титово ...","Сюкпря Пазонтень, кона путсь Титовонь седейсэ ...",3.0,ru
1,"Ибо если я снова созидаю, что разрушил, то сам...","Бути мон таго теинь сень, мезе разрушил, сестэ...",2.0,ru
2,И вот что рассказывают наши русские старики: «...,Вана мезе ёвтнить миненек рузонь атятне: зярдо...,4.0,ru
3,"Старшему имя Михаил, второму — Семен, третьему...","Покшонть лемезэ ульнесь Мишка, омбоценть — Сём...",4.5,ru
4,"Там будем, эрзяне, жить, Уж там построим мы те...","Тосо карматано, эрзят, эрямо, Уш тосо стявттан...",3.0,ru
5,ф) установление основ законодательства о труде;,ф) трудоустройствань закононь основатнень уста...,1.0,ru
6,Высший надзор за точным исполнением законов вс...,Весе Народной Комиссариаттнэнь ды сынст подвед...,3.0,ru
7,Участники игры мальчики и девочки 6 — 12 лет.,Налксицятне 6 — 12 иесэ цёрынеть ды тейтернеть.,4.0,ru
8,И сердце у тебя совсем другое…,Ды седееть тунь лия...,4.0,ru
9,— Ты это к чему? — осторожно осведомился Михалыч.,— Тон те мезекс? — чевтестэ чарькодсь Михалыч.,2.0,ru


In [ ]:
from transformers import AutoModelForSequenceClassification, AutoModelForPreTraining, AutoTokenizer

In [ ]:
mname = BASE_DIR + 'labse_erzya_v1'
tokenizer = AutoTokenizer.from_pretrained(mname)
model = AutoModelForPreTraining.from_pretrained(mname)

In [ ]:
def classify(text1, text2, model, tokenizer):
    batch = tokenizer(text1, text2, truncation=True, return_tensors='pt').to(model.device)
    with torch.inference_mode():
        out = torch.softmax(model(**batch).seq_relationship_logits, -1).cpu().numpy()[0, 1]
    return out

print(classify(manual_scores.original[0], manual_scores.translation[0], model, tokenizer))

0.9978003


In [ ]:
def get_dot(text1, text2, model, tokenizer):
    batch = tokenizer([text1, text2], truncation=True, padding=True, return_tensors='pt').to(model.device)
    with torch.inference_mode():
        embs = torch.nn.functional.normalize(model.bert(**batch).pooler_output)
    return torch.dot(embs[0], embs[1]).item()

print(get_dot(manual_scores.original[0], manual_scores.translation[0], model, tokenizer))
print(get_dot(manual_scores.original[0], manual_scores.original[0], model, tokenizer))
print(get_dot(manual_scores.original[0], manual_scores.translation[1], model, tokenizer))

0.8851594924926758
0.9999999403953552
0.1809203028678894


In [ ]:
manual_scores['s1'] = [classify(row.original, row.translation, model, tokenizer) for _, row in manual_scores.iterrows()]

In [ ]:
manual_scores['s2'] = [classify(row.translation, row.original, model, tokenizer) for _, row in manual_scores.iterrows()]

In [ ]:
manual_scores['s12'] = manual_scores['s2'] * manual_scores['s1']

In [ ]:
manual_scores['cossim'] = [get_dot(row.translation, row.original, model, tokenizer) for _, row in manual_scores.iterrows()]

In [ ]:
manual_scores.corr('spearman')

,score,s1,s2,s12,cossim
score,1.000000,-0.009539,0.128521,0.074552,0.282646
s1,-0.009539,1.000000,0.893596,0.928079,0.153202
s2,0.128521,0.893596,1.000000,0.985714,0.300493
s12,0.074552,0.928079,0.985714,1.000000,0.269458
cossim,0.282646,0.153202,0.300493,0.269458,1.000000


In [ ]:
manual_scores.groupby('source').corr('spearman')

score        s1        s2       s12    cossim
source                                                         
myv    score   1.000000  0.050138  0.157935  0.106543  0.626726
       s1      0.050138  1.000000  0.911765  0.948529  0.039216
       s2      0.157935  0.911765  1.000000  0.985294  0.161765
       s12     0.106543  0.948529  0.985294  1.000000  0.129902
       cossim  0.626726  0.039216  0.161765  0.129902  1.000000
ru     score   1.000000 -0.104360 -0.064775 -0.050381 -0.287889
       s1     -0.104360  1.000000  0.846154  0.874126  0.153846
       s2     -0.064775  0.846154  1.000000  0.986014  0.356643
       s12    -0.050381  0.874126  0.986014  1.000000  0.314685
       cossim -0.287889  0.153846  0.356643  0.314685  1.000000

In [ ]:
import scipy.stats

In [ ]:
scipy.stats.spearmanr(manual_scores[manual_scores.source=='myv'].score, manual_scores[manual_scores.source=='myv'].s12)

SpearmanrResult(correlation=0.10654339832432477, pvalue=0.6840146618612767)

In [ ]:
scipy.stats.spearmanr(manual_scores[manual_scores.source=='ru'].score, manual_scores[manual_scores.source=='ru'].s12)

SpearmanrResult(correlation=-0.05038057699354522, pvalue=0.8764351849316754)

In [ ]:
scipy.stats.spearmanr(manual_scores[manual_scores.source=='myv'].score, manual_scores[manual_scores.source=='myv'].cossim)

SpearmanrResult(correlation=0.6267258724960281, pvalue=0.0070970035693212205)

In [ ]:
scipy.stats.spearmanr(manual_scores[manual_scores.source=='ru'].score, manual_scores[manual_scores.source=='ru'].cossim)

SpearmanrResult(correlation=-0.28788901139168704, pvalue=0.3642014741833626)

Correlation is no statistically significant. Either the model is undertrained for the translation detection loss, or the corrupted training samples are not representative of the real translation errors.

In [ ]:
manual_scores

,original,translation,score,source,s1,s2,s12
0,"Благодарение Богу, вложившему в сердце Титово ...","Сюкпря Пазонтень, кона путсь Титовонь седейсэ ...",3.0,ru,0.997800,0.994707,0.992519
1,"Ибо если я снова созидаю, что разрушил, то сам...","Бути мон таго теинь сень, мезе разрушил, сестэ...",2.0,ru,0.995986,0.996814,0.992813
2,И вот что рассказывают наши русские старики: «...,Вана мезе ёвтнить миненек рузонь атятне: зярдо...,4.0,ru,0.958871,0.936734,0.898207
3,"Старшему имя Михаил, второму — Семен, третьему...","Покшонть лемезэ ульнесь Мишка, омбоценть — Сём...",4.5,ru,0.988938,0.984304,0.973415
4,"Там будем, эрзяне, жить, Уж там построим мы те...","Тосо карматано, эрзят, эрямо, Уш тосо стявттан...",3.0,ru,0.983811,0.986435,0.970466
5,ф) установление основ законодательства о труде;,ф) трудоустройствань закононь основатнень уста...,1.0,ru,0.991507,0.992002,0.983577
6,Высший надзор за точным исполнением законов вс...,Весе Народной Комиссариаттнэнь ды сынст подвед...,3.0,ru,0.974168,0.979692,0.954385
7,Участники игры мальчики и девочки 6 — 12 лет.,Налксицятне 6 — 12 иесэ цёрынеть ды тейтернеть.,4.0,ru,0.995447,0.996098,0.991563
8,И сердце у тебя совсем другое…,Ды седееть тунь лия...,4.0,ru,0.999214,0.999179,0.998394
9,— Ты это к чему? — осторожно осведомился Михалыч.,— Тон те мезекс? — чевтестэ чарькодсь Михалыч.,2.0,ru,0.995426,0.997937,0.993373


# The hassle with tokenizing

In [ ]:
base_model_path = '/gd/MyDrive/models/myv/mbart-large-51-myv-raw'

In [ ]:
def fix_tokenizer(tokenizer):
    old_len = len(tokenizer) - int('myv_XX' in tokenizer.added_tokens_encoder)
    tokenizer.lang_code_to_id['myv_XX'] = old_len-1
    tokenizer.id_to_lang_code[old_len-1] = 'myv_XX'
    tokenizer.fairseq_tokens_to_ids["<mask>"] = len(tokenizer.sp_model) + len(tokenizer.lang_code_to_id) + tokenizer.fairseq_offset

    tokenizer.fairseq_tokens_to_ids.update(tokenizer.lang_code_to_id)
    tokenizer.fairseq_ids_to_tokens = {v: k for k, v in tokenizer.fairseq_tokens_to_ids.items()}
    if 'myv_XX' not in tokenizer._additional_special_tokens:
        tokenizer._additional_special_tokens.append('myv_XX')
    tokenizer.added_tokens_encoder = {}

In [ ]:
tok1 = MBart50Tokenizer.from_pretrained(base_model_path)
print(len(tok1), tok1.vocab_size, len(tok1.sp_model), len(tok1.lang_code_to_id), len(tok1.added_tokens_encoder))
fix_tokenizer(tok1)
print(len(tok1), tok1.vocab_size, len(tok1.sp_model), len(tok1.lang_code_to_id), len(tok1.added_tokens_encoder))

269545 269545 269491 52 0
269546 269546 269491 53 0


In [ ]:
tok1.added_tokens_encoder

{}

In [ ]:
tok1.src_lang = 'myv_XX'
print(tok1.convert_tokens_to_ids('myv_XX'))

269544


In [ ]:
tok1('Ды вана мезе ёвтнить минек эрзянь атятне: «Зярдояк те велесэнть арасель, тосо оврагасо ульнесть розбойникть; сынст ульнесть землянкаст.')

{'input_ids': [269544, 95080, 6, 63689, 250041, 6, 254809, 250070, 250063, 257453, 12, 94, 256005, 17931, 1960, 252028, 250586, 4, 690, 6346, 407, 59667, 680, 6346, 250136, 268130, 2549, 1117, 74, 250074, 250136, 68605, 13405, 2225, 5, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
t = MBart50Tokenizer.from_pretrained(BASE_DIR + 'mbart-large-51-myv-mul-v1')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
tok2 = MBart50Tokenizer.from_pretrained(BASE_DIR + 'mbart-large-51-myv-mul-v1')
print(len(tok2), tok2.vocab_size, len(tok2.sp_model), len(tok2.lang_code_to_id), len(tok2.added_tokens_encoder))
fix_tokenizer(tok2)
print(len(tok2), tok2.vocab_size, len(tok2.sp_model), len(tok2.lang_code_to_id), len(tok2.added_tokens_encoder))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


269546 269545 269491 52 1
269546 269546 269491 53 0


In [ ]:
tok2.src_lang = 'myv_XX'
print(tok2.convert_tokens_to_ids('myv_XX'))

269544


In [ ]:
tok2('Ды вана мезе ёвтнить минек эрзянь атятне: «Зярдояк те велесэнть арасель, тосо оврагасо ульнесть розбойникть; сынст ульнесть землянкаст.')

{'input_ids': [269544, 95080, 6, 63689, 250041, 6, 254809, 250070, 250063, 257453, 12, 94, 256005, 17931, 1960, 252028, 250586, 4, 690, 6346, 407, 59667, 680, 6346, 250136, 268130, 2549, 1117, 74, 250074, 250136, 68605, 13405, 2225, 5, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

It turned out that for some reason `added_tokens_encoder` of a tokenizer were were influenced before. We fixed this now, and make a round-trip translation for the third time now. 

# What if we translate via Russian?

1. myv -> ru
2. ru-> en/fi, fi/en -> ru
3. ru-> myv

In [ ]:
with open('/gd/MyDrive/datasets/nlp/myv_corpus/myv-en-ud-test.json', 'r') as f:
    test_myv_en = json.load(f)
with open('/gd/MyDrive/datasets/nlp/myv_corpus/myv-fi-ud-test.json', 'r') as f:
    test_myv_fi = json.load(f)

len(test_myv_en), len(test_myv_fi)

(221, 155)

In [ ]:
test_myv_en[0]

['Нармуньковсто сёлмовтнень уш а токшить — истя жо пежеть.',
 'In the Bird Moon no one touches the winged ones any more, that too would be breaking a taboo.']

In [ ]:
import fasttext
langid_model = fasttext.load_model(BASE_DIR + 'lid.323.ftz')

In [ ]:
def get_mean_lang_score(text, lang='myv', k=300, max_score=0.3):
    words = text.split() + [text]
    res = []
    for langs, scores in zip(*langid_model.predict(words, k=k)):
        d = dict(zip([l[9:] for l in langs], scores))
        score = min(d.get(lang, 0), max_score) / max_score
        res.append(score)
    # print(res)
    return np.mean(res)

print(get_mean_lang_score('как дела'))
print(get_mean_lang_score('Овто патяй'))
print(get_mean_lang_score('Ардсть, ардсть, каршозост моли верьгиз ды кевкстни: — Овто патяй, ков молят?'))

3.3362521207891405e-05
0.007156715593939427
0.2334256109004342


In [ ]:
def translate(text, model, tokenizer, src='ru_RU', trg='myv_XX', max_length='auto', num_beams=3, repetition_penalty=5.0, train_mode=False, n_out=None, **kwargs):
    tokenizer.src_lang = src
    encoded = tokenizer(text, return_tensors="pt", truncation=True, max_length=1024)
    if max_length == 'auto':
        max_length = int(32 + 1.5 * encoded.input_ids.shape[1])
    if train_mode:
        model.train()
    else:
        model.eval()
    generated_tokens = model.generate(
        **encoded.to(model.device),
        forced_bos_token_id=tokenizer.lang_code_to_id[trg], 
        max_length=max_length, 
        num_beams=num_beams,
        repetition_penalty=repetition_penalty,
        # early_stopping=True,
        num_return_sequences=n_out or 1,
        **kwargs
    )
    out = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
    if isinstance(text, str) and n_out is None:
        return out[0]
    return out

In [ ]:
def translate_rerank(
    text, model, tokenizer, src='ru_RU', trg='myv_XX', max_length='auto', num_beams=3, repetition_penalty=5.0, train_mode=False, 
    n=5, diversity_penalty=3.0, lang='myv', max_score=0.3, order_penalty=0.01,
    verbose=False,
    **kwargs
):
    texts = translate(
        text, model, tokenizer, src, trg, 
        max_length=max_length, train_mode=train_mode, repetition_penalty=repetition_penalty, 
        num_beams=n,
        num_beam_groups=n, 
        diversity_penalty=diversity_penalty, 
        n_out=n, 
        **kwargs
    )
    scores = [get_mean_lang_score(t, lang=lang, max_score=max_score) for t in texts]
    pen_scores = scores - order_penalty * np.arange(n)
    if verbose:
        print(texts)
        print(scores)
        print(pen_scores)
    return texts[np.argmax(pen_scores)]
    

## Forcing the language

1. myv -> ru

In [ ]:
mname = BASE_DIR + 'mbart-large-51-myv-mul-v1'
model = MBartForConditionalGeneration.from_pretrained(mname).cuda()

tokenizer = MBart50Tokenizer.from_pretrained(mname)
fix_tokenizer(tokenizer)
cleanup()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
test_myv_en_myv2ru = [translate_rerank(myv, model, tokenizer, 'myv_XX', 'ru_RU', num_beams=5, repetition_penalty=5.0, lang='ru') for myv, en in tqdm(test_myv_en)]

  0%|          | 0/221 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/generation_beam_search.py:198: UserWarning: Passing `max_length` to BeamSearchScorer is deprecated and has no effect. `max_length` should be passed directly to `beam_search(...)`, `beam_sample(...)`, or `group_beam_search(...)`.
  "Passing `max_length` to BeamSearchScorer is deprecated and has no effect. "


In [ ]:
test_myv_fi_myv2ru = [translate_rerank(myv, model, tokenizer, 'myv_XX', 'ru_RU', num_beams=5, repetition_penalty=5.0, lang='ru') for myv, fi in tqdm(test_myv_fi)]

  0%|          | 0/155 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/generation_beam_search.py:198: UserWarning: Passing `max_length` to BeamSearchScorer is deprecated and has no effect. `max_length` should be passed directly to `beam_search(...)`, `beam_sample(...)`, or `group_beam_search(...)`.
  "Passing `max_length` to BeamSearchScorer is deprecated and has no effect. "


2. en, fi <-> ru

In [ ]:
mname =  'facebook/mbart-large-50-many-to-many-mmt'
model = MBartForConditionalGeneration.from_pretrained(mname).cuda()
tokenizer = MBart50Tokenizer.from_pretrained(mname)
cleanup()

In [ ]:
test_myv_en_myv2ru_en = [translate_rerank(t, model, tokenizer, 'ru_RU', 'en_XX', num_beams=5, repetition_penalty=5.0, lang='en') for t in tqdm(test_myv_en_myv2ru)]

  0%|          | 0/221 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/generation_beam_search.py:198: UserWarning: Passing `max_length` to BeamSearchScorer is deprecated and has no effect. `max_length` should be passed directly to `beam_search(...)`, `beam_sample(...)`, or `group_beam_search(...)`.
  "Passing `max_length` to BeamSearchScorer is deprecated and has no effect. "


In [ ]:
test_myv_fi_myv2ru_fi = [translate_rerank(t, model, tokenizer, 'ru_RU', 'fi_FI', num_beams=5, repetition_penalty=5.0, lang='fi') for t in tqdm(test_myv_fi_myv2ru)]

  0%|          | 0/155 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/generation_beam_search.py:198: UserWarning: Passing `max_length` to BeamSearchScorer is deprecated and has no effect. `max_length` should be passed directly to `beam_search(...)`, `beam_sample(...)`, or `group_beam_search(...)`.
  "Passing `max_length` to BeamSearchScorer is deprecated and has no effect. "


In [ ]:
test_myv_en_en2ru = [translate_rerank(en, model, tokenizer, 'en_XX', 'ru_RU', num_beams=5, repetition_penalty=5.0, lang='ru') for myv, en in tqdm(test_myv_en)]
test_myv_fi_fi2ru = [translate_rerank(fi, model, tokenizer, 'fi_FI', 'ru_RU', num_beams=5, repetition_penalty=5.0, lang='ru') for myv, fi in tqdm(test_myv_fi)]

  0%|          | 0/221 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/generation_beam_search.py:198: UserWarning: Passing `max_length` to BeamSearchScorer is deprecated and has no effect. `max_length` should be passed directly to `beam_search(...)`, `beam_sample(...)`, or `group_beam_search(...)`.
  "Passing `max_length` to BeamSearchScorer is deprecated and has no effect. "


  0%|          | 0/155 [00:00<?, ?it/s]

3. ru -> myv

In [ ]:
mname = BASE_DIR + 'mbart-large-51-mul-myv-v1'
model = MBartForConditionalGeneration.from_pretrained(mname).cuda()

In [ ]:
tokenizer = MBart50Tokenizer.from_pretrained(mname)
fix_tokenizer(tokenizer)
cleanup()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
test_myv_en_en2ru_myv = [translate_rerank(t, model, tokenizer, 'ru_RU', 'myv_XX', num_beams=5, repetition_penalty=5.0, lang='myv') for t in tqdm(test_myv_en_en2ru)]
test_myv_fi_fi2ru_myv = [translate_rerank(t, model, tokenizer, 'ru_RU', 'myv_XX', num_beams=5, repetition_penalty=5.0, lang='myv') for t in tqdm(test_myv_fi_fi2ru)]

  0%|          | 0/221 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/generation_beam_search.py:198: UserWarning: Passing `max_length` to BeamSearchScorer is deprecated and has no effect. `max_length` should be passed directly to `beam_search(...)`, `beam_sample(...)`, or `group_beam_search(...)`.
  "Passing `max_length` to BeamSearchScorer is deprecated and has no effect. "


  0%|          | 0/155 [00:00<?, ?it/s]

In [ ]:
from sacrebleu import CHRF, BLEU
chrf_calc = CHRF(word_order=2)
bleu_calc = BLEU()

In [ ]:
print('en-myv', chrf_calc.corpus_score(test_myv_en_en2ru_myv, [[myv for myv, en in test_myv_en]]).score, bleu_calc.corpus_score(test_myv_en_en2ru_myv, [[myv for myv, en in test_myv_en]]).score)
print('fi-myv', chrf_calc.corpus_score(test_myv_fi_fi2ru_myv, [[myv for myv, fi in test_myv_fi]]).score, bleu_calc.corpus_score(test_myv_fi_fi2ru_myv, [[myv for myv, fi in test_myv_fi]]).score)
print('myv-en', chrf_calc.corpus_score(test_myv_en_myv2ru_en, [[en for myv, en in test_myv_en]]).score, bleu_calc.corpus_score(test_myv_en_myv2ru_en, [[en for myv, en in test_myv_en]]).score)
print('myv-fi', chrf_calc.corpus_score(test_myv_fi_myv2ru_fi, [[fi for myv, fi in test_myv_fi]]).score, bleu_calc.corpus_score(test_myv_fi_myv2ru_fi, [[fi for myv, fi in test_myv_fi]]).score)

en-myv 21.834659426782945 1.917292419861903
fi-myv 19.803759764448543 2.504217528397886
myv-en 19.830213490149514 2.784295023759409
myv-fi 14.092061314826484 0.9842743948486357


## Not forcing the language

1. myv -> ru

In [ ]:
mname = BASE_DIR + 'mbart-large-51-myv-mul-v1'
model = MBartForConditionalGeneration.from_pretrained(mname).cuda()

tokenizer = MBart50Tokenizer.from_pretrained(mname)
fix_tokenizer(tokenizer)
cleanup()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
test_myv_en_myv2ru2 = [translate(myv, model, tokenizer, 'myv_XX', 'ru_RU', num_beams=5, repetition_penalty=5.0) for myv, en in tqdm(test_myv_en)]

  0%|          | 0/221 [00:00<?, ?it/s]

In [ ]:
test_myv_fi_myv2ru2 = [translate(myv, model, tokenizer, 'myv_XX', 'ru_RU', num_beams=5, repetition_penalty=5.0) for myv, fi in tqdm(test_myv_fi)]

  0%|          | 0/155 [00:00<?, ?it/s]

2. en, fi <-> ru

In [ ]:
mname =  'facebook/mbart-large-50-many-to-many-mmt'
model = MBartForConditionalGeneration.from_pretrained(mname).cuda()
tokenizer = MBart50Tokenizer.from_pretrained(mname)
cleanup()

In [ ]:
test_myv_en_myv2ru_en2 = [translate(t, model, tokenizer, 'ru_RU', 'en_XX', num_beams=5, repetition_penalty=5.0) for t in tqdm(test_myv_en_myv2ru2)]

  0%|          | 0/221 [00:00<?, ?it/s]

In [ ]:
test_myv_fi_myv2ru_fi2 = [translate(t, model, tokenizer, 'ru_RU', 'fi_FI', num_beams=5, repetition_penalty=5.0) for t in tqdm(test_myv_fi_myv2ru2)]

  0%|          | 0/155 [00:00<?, ?it/s]

In [ ]:
test_myv_en_en2ru = [translate(en, model, tokenizer, 'en_XX', 'ru_RU', num_beams=5, repetition_penalty=5.0) for myv, en in tqdm(test_myv_en)]
test_myv_fi_fi2ru = [translate(fi, model, tokenizer, 'fi_FI', 'ru_RU', num_beams=5, repetition_penalty=5.0) for myv, fi in tqdm(test_myv_fi)]

  0%|          | 0/221 [00:00<?, ?it/s]

  0%|          | 0/155 [00:00<?, ?it/s]

3. ru -> myv

In [ ]:
mname = BASE_DIR + 'mbart-large-51-mul-myv-v1'
model = MBartForConditionalGeneration.from_pretrained(mname).cuda()

In [ ]:
tokenizer = MBart50Tokenizer.from_pretrained(mname)
fix_tokenizer(tokenizer)
cleanup()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
test_myv_en_en2ru_myv2 = [translate(t, model, tokenizer, 'ru_RU', 'myv_XX', num_beams=5, repetition_penalty=5.0) for t in tqdm(test_myv_en_en2ru)]
test_myv_fi_fi2ru_myv2 = [translate(t, model, tokenizer, 'ru_RU', 'myv_XX', num_beams=5, repetition_penalty=5.0) for t in tqdm(test_myv_fi_fi2ru)]

  0%|          | 0/221 [00:00<?, ?it/s]

  0%|          | 0/155 [00:00<?, ?it/s]

In [ ]:
test_myv_en_en2ru[:15]

['В Птичьей Луне никто больше не трогает крыльев, то тоже было бы нарушение табу.',
 'Он взглянул с тревоги, надеясь, что крыша не впадет в него.',
 'Один за другим он встречается с маленькими рощами, где он может отдохнуть во время своего путешествия.',
 'Солдат, вылечившийся от ран, вернулся из войны, его жена испугалась, она начала кричать «О мой Бог!».',
 'Что вы думаете?',
 'Он взял небольшую мелкую кожу из ковчега, распыл белый песок с берега реки на зуб и протянул его между пальцами, пока он не сделал отверстие через зуб.',
 'Его мать, должно быть, решила позволить ему смириться и только теперь, после того как она сама зажегла огонь и взяла воду, она послала Лопу разбудить его.',
 'Деревня простиралась вдоль холмов, образуя бассейн.',
 'Он тоже был сильным человеком.',
 'Мы самые счастливые в мире, товарищи.',
 'Все хорошо.',
 'Не ожидайте, что леди леса сделает какие-либо исключения.',
 '— Посмотрите, что у меня есть.',
 'Девушка выглядела как её отец.',
 'Они он сам пошел в од

In [ ]:
test_myv_en_en2ru_myv2[:20]

['Птичье Луна кияк больше а трогает крыльев, то тоже было бы нарушение табу.',
 'Сон тандадозь варштась, ансяк арсесь: кудозо а понги тензэ.',
 'Вейке омбоце мельга сон вастови вишка роща марто, косо сон может отдохнуть во время своего путешествия.',
 'солдатось, кона пичкась ранонзо эйстэ, велявтсь уш войнасто, низэ тандадсь, кармась пижнеме: «О мой Бог!».',
 'Мезе арсетядо?',
 'Сон саизе а покш, вишкине коцтонть лисьмасто, вачкодинзе ашо пес леенть чиреванзо пеенть лангс ды венстявтызе сонзэ суротнень юткова, зярс эзь тее пеень трокс отверстия.',
 'Сонзэ авазо, нама, арсесь кальдяв тензэ мириться ды ансяк ней, сонсь кирвазтизе толонть ды саизе ведентень, сон кучизе Лопу сонзэ сыргозтеме.',
 'Велесь простирась пандо ланга, теевсь бассейнакс.',
 'Сон жо ульнесь виев ломань.',
 'Минь сехте уцяскавт, ялгат.',
 'Весе вадрясто.',
 'Иля учо, вирень леди кодамояк исключения.',
 '— Вант, мезе ули.',
 'Тейтересь марявсь тетянзо кондямо.',
 'Сынь сонсь тусь вейке модаваряс.',
 'Кечаень туртов м

In [ ]:
from sacrebleu import CHRF, BLEU
chrf_calc = CHRF(word_order=2)
bleu_calc = BLEU()

In [ ]:
print('en-myv', chrf_calc.corpus_score(test_myv_en_en2ru_myv2, [[myv for myv, en in test_myv_en]]).score, bleu_calc.corpus_score(test_myv_en_en2ru_myv2, [[myv for myv, en in test_myv_en]]).score)
print('fi-myv', chrf_calc.corpus_score(test_myv_fi_fi2ru_myv2, [[myv for myv, fi in test_myv_fi]]).score, bleu_calc.corpus_score(test_myv_fi_fi2ru_myv2, [[myv for myv, fi in test_myv_fi]]).score)
print('myv-en', chrf_calc.corpus_score(test_myv_en_myv2ru_en2, [[en for myv, en in test_myv_en]]).score, bleu_calc.corpus_score(test_myv_en_myv2ru_en2, [[en for myv, en in test_myv_en]]).score)
print('myv-fi', chrf_calc.corpus_score(test_myv_fi_myv2ru_fi2, [[fi for myv, fi in test_myv_fi]]).score, bleu_calc.corpus_score(test_myv_fi_myv2ru_fi2, [[fi for myv, fi in test_myv_fi]]).score)

en-myv 25.890317780325432 3.0363182731183724
fi-myv 23.50778586720546 2.7070909472763316
myv-en 23.83186958948613 5.593264667970031
myv-fi 16.05029821283373 1.0261582353576988


# Look at examples

In [ ]:
import json

In [ ]:
BASE_DIR = '/gd/MyDrive/models/myv/'

In [ ]:
with open('/gd/MyDrive/datasets/nlp/myv_corpus/dev1000.v1.json', 'r') as f:
    dev = json.load(f)

In [ ]:
with open(BASE_DIR + 'dev_myv_mul_predictions.json', 'r') as f:
    devt = json.load(f)

```
with open(BASE_DIR + 'dev_myv_mul_predictions.json', 'w') as f:
    json.dump({
        'en_myv': pred_en_myv,
        'fi_myv': pred_fi_myv,
        'myv_en': pred_myv_en,
        'myv_fi': pred_myv_fi,
        'en_myv_en': pred_en_myv_en,
        'fi_myv_fi': pred_fi_myv_fi,
        'myv_en_myv': pred_myv_en_myv,
        'myv_fi_myv': pred_myv_fi_myv,
        'myv_ru': dev_translated_ru,
        'ru_myv': dev_translated,
        'myv_ru_myv': dev_myv_round,
        'ru_myv_ru': dev_ru_round,
    }, f, ensure_ascii=False, indent=2)
```

In [ ]:
dev.keys()

dict_keys(['bible', 'tales', 'constitution', 'games', 'fiction', 'wiki'])

In [ ]:
ru_true = 'И вот что рассказывают наши русские старики: «Когда здесь не было этого села, в этом овраге были разбойники; у них были землянки.'

for i, pair in enumerate(dev['tales']):
    if pair[1] == ru_true:
        myv_true = pair[0]
        print(myv_true)
        print(devt['ru_myv']['tales'][i])
        print(devt['myv_ru']['tales'][i])

Вана мезе ёвтнить миненек рузонь атятне: зярдо велесь тесэ арасель, се латксонть эрясть розбойникть, эрясть землянкасо.
Ды вана мезе ёвтнить минек эрзянь атятне: «Зярдояк те велесэнть арасель, тосо оврагасо ульнесть розбойникть; сынст ульнесть землянкаст.
Вот что нам говорят русские старцы: когда деревня здесь не была, то там жилибойники, жили на земле.


In [ ]:
with open('/gd/MyDrive/datasets/nlp/myv_corpus/myv-en-ud-dev.json', 'r') as f:
    dev_myv_en = json.load(f)
with open('/gd/MyDrive/datasets/nlp/myv_corpus/myv-fi-ud-dev.json', 'r') as f:
    dev_myv_fi = json.load(f)

In [ ]:
import random


ss = sorted({p[0] for p in dev_myv_en}.intersection({p[0] for p in dev_myv_fi}))
print(len(ss))
random.seed(3)
myv = random.choice(ss)
print(myv)

69
Кода авазо, анокстась лапужа кирькст, истя жо педявтнинзе, валаськавтнинзе педявтома таркатнень начко кедьсэ.


In [ ]:
for i, pair in enumerate(dev_myv_en):
    if pair[0] == myv:
        en = pair[1]
        print(en)
        print(myv)
        print(devt['en_myv'][i])
        print(devt['myv_en'][i])

Like his mother, he prepared flat rings, and stuck them onto the patty in the same way, and smoothed out the seams with his wet hands.
Кода авазо, анокстась лапужа кирькст, истя жо педявтнинзе, валаськавтнинзе педявтома таркатнень начко кедьсэ.
Кода аванзо, сон анокстыль валаня суркст, теке ладсо педявтызе сынст пацьказонзо ды вадяшась кедень летькенть марто.
Like his mother, he prepared flat circles, and also filled the canvas with a needle.


In [ ]:
for i, pair in enumerate(dev_myv_fi):
    if pair[0] == myv:
        en = pair[1]
        print(en)
        print(myv)
        print(devt['fi_myv'][i])
        print(devt['myv_fi'][i])

Samalla tavalla kuin äitinsä Ketšai valmisti litteitä rinkuloita liitti ne samalla tavalla, ja siloitti liitoksen märällä kädellä.
Кода авазо, анокстась лапужа кирькст, истя жо педявтнинзе, валаськавтнинзе педявтома таркатнень начко кедьсэ.
Истя жо, кода авазо Кетшай анокстыль лаҥгсо кевпанть, сон солодиль сынст теке ладсо ды солодиль эйсэст кедьлапушкасо.
Kuten äiti, valmistelee tasa-alaiset kentät, myös venytetään, lyödään venyttäjän käsillä.
